In [1]:
import os

In [2]:
import numpy as np
import random
import datetime

In [3]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import Arrow, NormalHead, OpenHead, VeeHead
from bokeh.models import Label
from bokeh.models import Span
from bokeh.embed import components


In [4]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats


In [5]:
import pandas as pd
DF = pd.read_csv('data/seven_day_county.csv')
DF['date'] = pd.to_datetime(DF['date'])

#duncan = pd.read_csv('data/Duncan.csv')
DF.head()
df_yakima = DF[(DF['state'] == 'Washington') & (DF['county'] == 'Yakima')]
#df_yakima = df_yakima[(df_yakima['date']< datetime.datetime(2020, 6, 20))]
df_yakima.head()

,state,county,date,new_deaths,new_cases,population,rucc_2013
272082,Washington,Yakima,2020-03-13,0,0,250873,3
272083,Washington,Yakima,2020-03-14,0,1,250873,3
272084,Washington,Yakima,2020-03-15,0,1,250873,3
272085,Washington,Yakima,2020-03-16,0,0,250873,3
272086,Washington,Yakima,2020-03-17,0,1,250873,3


In [6]:
def output_graphs(script, div, text, 
                 home_page_dir = '/home/henry/projects/covid19/home_page/'):
    with open(os.path.join(home_page_dir, 'script'), 'w') as write_obj:
              write_obj.write(script)
    with open(os.path.join(home_page_dir, 'div'), 'w') as write_obj:
              write_obj.write(div)
    with open(os.path.join(home_page_dir, 'text.txt'), 'w') as write_obj:
              write_obj.write(text)


In [7]:
import sklearn.linear_model as lm


In [8]:
output_notebook()

Loading BokehJS ...

In [9]:
#linear_model = lm.LinearRegression()
dates_df = df_yakima[(df_yakima['date'] >= datetime.datetime(2020, 5, 28))
                    
                    ]
X = list(range(1, len(dates_df['new_cases']) + 1))
dates_df = dates_df.assign(X = X)
model = smf.ols(formula='new_cases ~ X' , data=dates_df)
res = model.fit()
print(res.pvalues)
a, b1 = res.params
y_fit = [a + b1 * x for x in X]
y_fit
labels = df_yakima['date'].tolist()
labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
nums = df_yakima['new_cases'].rolling(7).mean()
p = figure(x_axis_type = 'datetime', title = 'Yakima Mask Usage', 
                 plot_width = 650 , plot_height = 650, y_range = None)
p.vbar(x=labels, top=nums, line_width = 5, width = .9) 
sec_date = dates_df['date'].tolist()
p.line(x = sec_date, y = y_fit, line_width = 3, color = 'red')
citation = Label(x=datetime.datetime(2020, 4, 15), y=150, 
                 text='Mask Usage Doubled', 
                 border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
p.add_layout(Arrow(end=OpenHead(line_color="black", line_width=1),
                   x_start=datetime.datetime(2020, 5, 1), y_start=150, 
                   x_end=datetime.datetime(2020, 5, 25), y_end=100))
p.add_layout(citation)
p.yaxis.axis_label = 'cases'


show(p)

Intercept    3.618377e-08
X            2.072936e-01
dtype: float64


In [10]:
p2 = figure(x_axis_type = 'datetime', title = 'Yakima Mask Usage', 
                 plot_width = 650 , plot_height = 650, y_range = None)
"""Example of a bar plot"""
#bar 
labels = ['predicted', 'actual']
nums = [0, 826]
p2 = figure(x_range=labels,  title='IHME Prediction')
p2.vbar(x=labels, top=nums, width=0.5)
p2.xgrid.grid_line_color = None
p2.y_range.start = 0

show(p2)


In [11]:
DF.head()
df_king = DF[(DF['state'] == 'Washington') & (DF['county'] == 'King')]
#df_king= df_king[(df_king['date']< datetime.datetime(2020, 6, 20))]
df_king.head()

,state,county,date,new_deaths,new_cases,population,rucc_2013
269361,Washington,King,2020-02-29,1,3,2252782,1
269362,Washington,King,2020-03-01,2,7,2252782,1
269363,Washington,King,2020-03-02,3,4,2252782,1
269364,Washington,King,2020-03-03,4,7,2252782,1
269365,Washington,King,2020-03-04,1,11,2252782,1


In [12]:
dates_after_open = df_king[(df_king['date'] >= datetime.datetime(2020, 6, 5) + datetime.timedelta(days=5))                  ]
X = list(range(1, len(dates_after_open['new_cases']) + 1))
dates_after_open = dates_after_open.assign(X = X)
model = smf.ols(formula='new_cases ~ X' , data=dates_after_open)
res = model.fit()
print(res.pvalues)
a, b1 = res.params
y_fit = [a + b1 * x for x in X]
dates_after_protest = df_king[(df_king['date'] >= datetime.datetime(2020, 5, 28) + datetime.timedelta(days=5))                  ]
X = list(range(1, len(dates_after_protest['new_cases']) + 1))
dates_after_protest = dates_after_protest.assign(X = X)
model = smf.ols(formula='new_cases ~ X' , data=dates_after_protest)
res = model.fit()
print(res.pvalues)
a, b1 = res.params
y_fit_2 = [a + b1 * x for x in X]
labels = df_king['date'].tolist()
labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
nums = df_king['new_cases'].rolling(3).mean()
nums2 = df_king['new_cases'].rolling(3).mean().tolist()
opening_start = Span(location=datetime.datetime(2020, 6, 5) + datetime.timedelta(days = 5),
                              dimension='height', line_color='green',
                              line_dash='dashed', line_width=3, )
protest_start = Span(location=datetime.datetime(2020, 5, 28) + datetime.timedelta(days = 5),
                              dimension='height', line_color='red',
                              line_dash='dashed', line_width=3, )
mask_start = Span(location=datetime.datetime(2020, 6, 26) + datetime.timedelta(days = 5),
                              dimension='height', line_color='blue',
                              line_dash='dashed', line_width=3, )
p = figure(x_axis_type = 'datetime', title = 'King', 
                 plot_width = 650 , plot_height = 650, y_range = None)
p.vbar(x=labels, top=nums, line_width = 5, width = .9)
#p.add_layout(opening_start)
#p.add_layout(protest_start)
p.add_layout(mask_start)
p.line(x = dates_after_open['date'], y = y_fit, color = 'green', line_width = 2, legend_label = 'Since Opening')
p.line(x =  dates_after_protest['date'], y = y_fit_2, color = 'red', line_width = 2, legend_label = 'Since Protest')

show(p)

Intercept    0.004121
X            0.000004
dtype: float64
Intercept    4.614662e-02
X            5.352364e-09
dtype: float64


In [13]:
def california_mandate():
    DF2 = pd.read_csv('data/states.csv')
    DF2['date'] = pd.to_datetime(DF2['date'])
    df_cal = DF2[(DF2['state'] == 'California') & (DF2['date'] >= datetime.datetime(2020, 6, 5))]
    dates_after_mask = df_cal[(df_cal['date'] >= datetime.datetime(2020, 6, 18) + datetime.timedelta(days=5))                    ]
    X = list(range(1, len(dates_after_mask['cases']) + 1))
    dates_after_open = dates_after_mask.assign(X = X)
    mask_start = Span(location=datetime.datetime(2020, 6, 18) + datetime.timedelta(days = 5),
                              dimension='height', line_color='green',
                              line_dash='dashed', line_width=3, )
    mandate = Span(location=datetime.datetime(2020, 6, 18) ,
                              dimension='height', line_color='red',
                              line_dash='dashed', line_width=3, )

    labels = df_cal['date'].tolist()
    labels1 = [x for x in labels if x > datetime.datetime(2020, 6, 18)]
    labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
    nums = df_cal['cases'].rolling(1).mean()
    p = figure(x_axis_type = 'datetime', title = 'California Mask Mandate', 
                 plot_width = 650 , plot_height = 650, y_range = None)



    dd = list(zip(labels, nums))
    incc = [x for x in dd if x[0] >  datetime.datetime(2020, 6, 18) 
        and x[0] <= datetime.datetime(2020, 6, 18) + datetime.timedelta(days = 5)
      ]
    bef = [x for x in dd if x[0] <= datetime.datetime(2020, 6, 18)]
    aff = [x for x in dd if x[0] > datetime.datetime(2020, 6, 18) + datetime.timedelta(days = 5)]

    p.vbar(x=[x[0] for x in bef], top=[x[1] for x in bef] , line_width = 5, width = .9, color = 'yellow',
       )
    p.vbar(x=[x[0] for x in incc], top=[x[1] for x in incc] , line_width = 5, width = .9, color = 'red',
       )
    p.vbar(x=[x[0] for x in aff], top=[x[1] for x in aff] , line_width = 5, width = .9, color = 'orange',
       )
    #p.add_layout(mask_start)
    #p.add_layout(mandate)
    m1 = np.mean([x[1] for x in bef][2:])
    m2 =np.mean([x[1] for x in aff])
    p_theory = stats.ttest_ind([x[1] for x in bef][2:],[x[1] for x in aff])[1]
    print(p_theory/2)
    mandate_l = Label(x=datetime.datetime(2020, 6, 4), y=4000, 
                 text='before mandate ({m:,})'.format(m = int(round(m1))), 
                 border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
    incubation = Label(x=datetime.datetime(2020, 6, 19), y=6600, 
                 text='Incubation', 
                 border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
    result = Label(x=datetime.datetime(2020, 7, 4), y=10000, 
                 text='Result ({m:,})'.format(m = int(round(m2))), 
                 border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
    dates_a = dates_after_open['date']
    p.add_layout(mandate_l)
    p.add_layout(incubation)
    p.add_layout(result)
    return p
p_california_mandate = california_mandate()
show(p_california_mandate)

1.557720886861972e-10


In [14]:
def make_washington_madate():
    DF3 = pd.read_csv('data/states.csv')
    DF3['date'] = pd.to_datetime(DF3['date'])
    df_wash = DF3[(DF3['state'] == 'Washington') & (DF3['date'] >= datetime.datetime(2020, 6, 1))]
    dates_after_mask = df_wash[(df_wash['date'] >= datetime.datetime(2020, 6, 26) + datetime.timedelta(days=5))                    ]
    X = list(range(1, len(dates_after_mask['cases']) + 1))
    dates_after_open = dates_after_mask.assign(X = X)
    mask_start = Span(location=datetime.datetime(2020, 6, 26) + datetime.timedelta(days = 5),
                              dimension='height', line_color='green',
                              line_dash='dashed', line_width=3, )
    mandate = Span(location=datetime.datetime(2020, 6, 26) ,
                              dimension='height', line_color='red',
                              line_dash='dashed', line_width=3, )

    labels = df_wash['date'].tolist()
    labels1 = [x for x in labels if x > datetime.datetime(2020, 6, 26)]
    labels = [datetime.datetime(x.year, x.month, x.day) for x in labels]
    nums = df_wash['cases'].rolling(3).mean()
    p = figure(x_axis_type = 'datetime', title = 'Washington Mask Mandate', 
                 plot_width = 650 , plot_height = 650, y_range = None)
    dd = list(zip(labels, nums))
    incc = [x for x in dd if x[0] >  datetime.datetime(2020, 6, 26) 
        and x[0] <= datetime.datetime(2020, 6, 26) + datetime.timedelta(days = 5)
      ]
    bef = [x for x in dd if x[0] <= datetime.datetime(2020, 6, 26)]
    aff = [x for x in dd if x[0] > datetime.datetime(2020, 6, 26) + datetime.timedelta(days = 5)]

    p.vbar(x=[x[0] for x in bef], top=[x[1] for x in bef] , line_width = 5, width = .9, color = 'yellow',
       )
    p.vbar(x=[x[0] for x in incc], top=[x[1] for x in incc] , line_width = 5, width = .9, color = 'red',
       )
    p.vbar(x=[x[0] for x in aff], top=[x[1] for x in aff] , line_width = 5, width = .9, color = 'orange',
       )
    m1 = np.mean([x[1] for x in bef][-7:])
    m2 =np.mean([x[1] for x in aff])
    p_theory = stats.ttest_ind([x[1] for x in bef][2:],[x[1] for x in aff])[1]
    print(p_theory/2)
    mandate_l = Label(x=datetime.datetime(2020, 6, 10), y=500, 
                 text='before mandate ({m:,})'.format(m = int(round(m1))), 
                 border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
    incubation = Label(x=datetime.datetime(2020, 6, 24), y=600, 
                 text='Incubation', 
                 border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
    result = Label(x=datetime.datetime(2020, 7, 5), y=750, 
                 text='Result ({m:,})'.format(m = int(round(m2))), 
                 border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0)
    dates_a = dates_after_open['date']
    p.add_layout(mandate_l)
    p.add_layout(incubation)
    p.add_layout(result)
    return p
p_wash_mandate = make_washington_madate()
show(p_wash_mandate)

1.2867591575113484e-10


In [15]:
script, div = components(p)
text = """Cases increased after the mask mandate. Mask Mandates alone are not enough to control a pandeic. (2020-07-18)"""
output_graphs(script, div, text = text)

In [76]:
def make_mask_graph(df):
    pass

def make_df(state, start_date):
    df = pd.read_csv('data/states.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df[(df['state'] == state) & (df['date'] >= start_date)]
    return df

def make_mask_graph(df, mask_start, title = None, plot_height = 450, plot_width = 450,
                   incubation_period = 5, window = 3):
    labels = df['date'].tolist()
    nums = df['cases'].rolling(window).mean()
    p = figure(x_axis_type = 'datetime', title = title, 
                 plot_width = plot_width , plot_height = plot_height, y_range = None)
    dd = list(zip(labels, nums))
    bef = [x for x in dd if x[0] <= mask_start]

    incc = [x for x in dd if x[0] >  mask_start 
        and x[0] <= mask_start + datetime.timedelta(days = incubation_period)
      ]
    aff = [x for x in dd if x[0] > mask_start + datetime.timedelta(days = incubation_period)]

    p.vbar(x=[x[0] for x in bef], top=[x[1] for x in bef] , line_width = 5, 
           width = .9, color = 'yellow', 
       )
    p.vbar(x=[x[0] for x in incc], top=[x[1] for x in incc] , line_width = 5, 
           width = .9, color = 'red', 
       )
    p.vbar(x=[x[0] for x in aff], top=[x[1] for x in aff] , line_width = 5, 
           width = .9, color = 'orange', 
       )
    p.legend.location = "top_center"
    return p

def do_mask_mandates(window = 3, plot_height = 450, 
                    plot_width = 450, ncols = 4):
    return gridplot([
            
            make_mask_graph(df = 
                make_df(state = 'California', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,6,18),
                    title = 'California', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ),   
                
            make_mask_graph(df = 
                make_df(state = 'Connecticut', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,4,20),
                    title = 'Connecticut', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ),
            make_mask_graph(df = 
                make_df(state = 'Delaware', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,4,28),
                    title = 'Delaware', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Hawaii', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,4,20),
                    title = 'Hawaii', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Illinois', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,5,1),
                    title = 'Illinois', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Kansas', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,7,3),
                    title = 'Kansas', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Kentucky', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,5,11),
                    title = 'Kentuckky', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
             make_mask_graph(df = 
                make_df(state = 'Maine', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,5,1),
                    title = 'Maine', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ),
             make_mask_graph(df = 
                make_df(state = 'Maryland', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,4,18),
                    title = 'Maryland', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
             make_mask_graph(df = 
                make_df(state = 'Massachusetts', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,5,6),
                    title = 'Massachusetts', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Michigan', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,6,18),
                    title = 'Michigan', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Nevada', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,6,24),
                    title = 'Nevada', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ),
            make_mask_graph(df = 
                make_df(state = 'New Jersey', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,4,8),
                    title = 'New Jersey', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ),
            make_mask_graph(df = 
                make_df(state = 'New Mexico', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,5,16),
                    title = 'New Mexico', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
             make_mask_graph(df = 
                make_df(state = 'New York', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,4,17),
                    title = 'New York', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ),
            make_mask_graph(df = 
                make_df(state = 'North Carolina', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,6,26),
                    title = 'North Corolina', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Oregon', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,7,1),
                    title = 'Oregeon', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Pennsylvania', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,4,19),
                    title = 'Pennsylvania', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ),
             make_mask_graph(df = 
                make_df(state = 'Rhode Island', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,5,18),
                    title = 'Rhode Island', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
             make_mask_graph(df = 
                make_df(state = 'Texas', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,7,3),
                    title = 'Texas', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Virginia', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,5,29),
                    title = 'Virginia', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'Washington', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,6,26),
                    title = 'Washington', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
            make_mask_graph(df = 
                make_df(state = 'West Virginia', 
                        start_date = datetime.datetime(2020,4, 1)),
                mask_start = datetime.datetime(2020,7,6),
                    title = 'West Virginia', plot_height = plot_height,
                                     plot_width = plot_width,
                            window = window,
               ), 
         ], 
         
         
         ncols = ncols)

show(do_mask_mandates(window = 7, plot_height = 250, plot_width = 250))
